In [31]:
import os.path
import sys
sys.path.insert(0, '../../')
from src.questions_construction.questions import FLUENT_TYPES_LIST
from src.questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from src.questions_construction.domains import DOMAIN_NAMES
from src.analysis.model_performances import * #gather_questions, TRANSPORTATION_DOMAINS, NON_TRANSPORTATION_DOMAINS
from src.common import *
import random
from collections import defaultdict
from copy import deepcopy
import itertools
from src.analysis.model_performances import *
import pandas as pd
from helpers import *

In [27]:
questions_dir = f'{DATA_PATH}/questions_m1'
test_ids = open_jsonl(f'{DATA_PATH}/dataset_ids.test.jsonl')
train_ids = open_jsonl(f'{DATA_PATH}/dataset_ids.train.jsonl')
questions_by_id = gather_questions(questions_dir)

In [28]:
for k, d in questions_by_id.items():
    if d['id'] in test_ids:
        d['split_type'] = 'test'
    else:
        d['split_type'] = 'train'

In [55]:
by_category = defaultdict(int)
save_key = 'dataset_split'
for d in questions_by_id.values():
    by_category[(d['question_category'], d['split_type'])]+=1

df_by_category = pd.DataFrame(index=sorted(QUESTION_CATEGORIES[:-1]), columns=['test', 'train'])
for k, v in by_category.items():
    df_by_category.loc[k[0], k[1]] = v
df_by_category

# by_category = defaultdict(int)
# save_key = 'dataset_split_by_answer_type'
# for d in questions_by_id.values():
#     by_category[(d['question_category'], d['split_type'], d['answer_type'])]+=1
# 
# df_by_category = pd.DataFrame(index=sorted(QUESTION_CATEGORIES[:-1]), columns=['test-true_false_answer', 'test-free_answer', 'train-true_false_answer', 'train-free_answer'])
# for k, v in by_category.items():
#     df_by_category.loc[k[0], k[1]+'-'+k[2]] = v
# df_by_category

,test-true_false_answer,test-free_answer,train-true_false_answer,train-free_answer
action_executability,240,119,4960,5081
effects,957,108,17443,3792
fluent_tracking,1889,952,32913,11048
hallucination,240,240,15560,26872
numerical_reasoning,240,120,25760,12880
object_tracking,1598,120,18990,2480
state_tracking,153,101,2447,3799


In [56]:
latex_table = to_latex_table(df_by_category, 'dataset split')

save_main_dir = f'{STATISTICS_PATH}.dataset_ids.test'
os.makedirs(os.path.join(save_main_dir, 'tables'), exist_ok=True)
with open(os.path.join(save_main_dir, 'tables', f'{save_key}.tex'), 'w') as f:
    f.write(latex_table)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  TO_PRETTY = {


In [39]:
by_category_by_fluents = defaultdict(int)
for d in questions_by_id.values():
    if d['fluent_type'] not in ('None', None):
        by_category_by_fluents[(d['question_category'], d['fluent_type'], d['split_type'])]+=1
        
columns = []
for c in ['base_fluents', 'persistent_fluents', 'derived_fluents', 'static_fluents']:
    columns.append(c+'-test')
    columns.append(c+'-train')
    
index = ['object_tracking', 'fluent_tracking', 'effects']
df = pd.DataFrame(index=index, columns=columns)
rows = []
for k, v in by_category_by_fluents.items():
    df.loc[k[0], k[1]+'-'+k[2]] = v
df

,base_fluents-test,base_fluents-train,persistent_fluents-test,persistent_fluents-train,derived_fluents-test,derived_fluents-train,static_fluents-test,static_fluents-train
object_tracking,382,3171,480,6724,264,3567,472,5528
fluent_tracking,707,7543,717,14883,712,10240,705,11295
effects,239,3761,240,4960,240,4360,238,4362


In [43]:
latex_table = to_latex_table(df, 'dataset split by fluents').replace('_', ' ')
save_key = 'fluent_split'
save_main_dir = f'{STATISTICS_PATH}.dataset_ids.test'
os.makedirs(os.path.join(save_main_dir, 'tables'), exist_ok=True)
with open(os.path.join(save_main_dir, 'tables', f'{save_key}.tex'), 'w') as f:
    f.write(latex_table)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
